In [2]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import mobilenet, resnet, densenet, efficientnet, vgg19
from tensorflow.keras.models import Model
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN, MeanShift, estimate_bandwidth, AffinityPropagation
import hdbscan
from sklearn import metrics
import umap

In [3]:
def load_and_preprocess_image(img_path, preprocess_input_func, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img = image.img_to_array(img)
    img = preprocess_input_func(img)
    return np.expand_dims(img, axis=0)

In [4]:
def create_feature_model(model_fn, preprocess_input_func):
    base_model = model_fn(weights='imagenet', include_top=False)
    feature_model = Model(inputs=base_model.input, outputs=base_model.output)
    return feature_model, preprocess_input_func

In [5]:
def extract_features_from_folder(folder_path, model_fn, preprocess_input_func):
    feature_model, preprocess_input = create_feature_model(model_fn, preprocess_input_func)
    features = []
    for img_name in os.listdir(folder_path):
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder_path, img_name)
            img = load_and_preprocess_image(img_path, preprocess_input)
            feature = feature_model.predict(img)
            features.append(feature.flatten())
    return np.array(features)

In [6]:
models = {
    'mobilenet': (mobilenet.MobileNet, mobilenet.preprocess_input),
    'resnet': (resnet.ResNet50, resnet.preprocess_input),
    'densenet': (densenet.DenseNet121, densenet.preprocess_input),
    'efficientnet': (efficientnet.EfficientNetB0, efficientnet.preprocess_input),
    'vgg19': (vgg19.VGG19, vgg19.preprocess_input),
}

img_dir = '/Users/kaavi/Documents/GitHub/term7cv/GUI_image_test/annotated_img/val_110_objects'

In [7]:
all_features = {}
for model_name, (model_fn, preprocess_input_func) in models.items():
    all_features[model_name] = extract_features_from_folder(img_dir, model_fn, preprocess_input_func)

1/1 [==============================] - 0s 90ms/step


In [8]:
reduced_features = {}
for model_name, features in all_features.items():
    features = StandardScaler().fit_transform(features)
    
    pca = PCA(n_components=0.9)
    reduced_features[(model_name, 'pca')] = pca.fit_transform(features)

    tsne = TSNE(n_components=2)
    reduced_features[(model_name, 'tsne')] = tsne.fit_transform(features)

    umap_reducer = umap.UMAP(n_neighbors=15, n_components=2)
    reduced_features[(model_name, 'umap')] = umap_reducer.fit_transform(features)


In [9]:
def calculate_accuracy(ground_truth_clusters, predicted_clusters):
    unique_predicted_clusters = len(set(predicted_clusters)) - (1 if -1 in predicted_clusters else 0)
    accuracy = unique_predicted_clusters / ground_truth_clusters
    return accuracy

In [10]:
results = {}
ground_truth_clusters = 10 

for (model_name, reduction_technique), features in reduced_features.items():
    # DBSCAN
    db = DBSCAN(eps=0.5, min_samples=5).fit(features)
    db_labels = db.labels_

    # HDBSCAN
    hdb = hdbscan.HDBSCAN(min_cluster_size=5).fit(features)
    hdb_labels = hdb.labels_

    # Mean Shift
    bandwidth = estimate_bandwidth(features, quantile=0.2)
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True).fit(features)
    ms_labels = ms.labels_

    # Affinity Propagation
    af = AffinityPropagation().fit(features)
    af_labels = af.labels_

    # Store results
    for algorithm, labels in [('dbscan', db_labels), ('hdbscan', hdb_labels), 
                              ('meanshift', ms_labels), ('affinity', af_labels)]:
        key = (model_name, reduction_technique, algorithm)
        silhouette = metrics.silhouette_score(features, labels) if len(set(labels)) > 1 else -1
        results[key] = {'silhouette_score': silhouette}

        # accuracy = calculate_accuracy(ground_truth_clusters, labels)
        # results[key] = {'silhouette_score': silhouette, 'accuracy': accuracy}

In [12]:
results_df = pd.DataFrame.from_dict(results, orient='index').reset_index()
results_df.columns = ['Model', 'Reduction Technique', 'Algorithm', 'Silhouette Score']  #, 'Accuracy'
results_df.sort_values(by='Silhouette Score', ascending=False, inplace=True)

results_df.reset_index(drop=True, inplace=True)
results_df.index += 1

print(results_df.head())

          Model Reduction Technique  Algorithm  Silhouette Score
1      densenet                umap   affinity          0.441988
2      densenet                umap  meanshift          0.419943
3        resnet                umap   affinity          0.399372
4        resnet                tsne   affinity          0.399105
5  efficientnet                umap   affinity          0.398480
